In [1]:
using ParserCombinator
using Base.Test
using AutoHashEquals
using AbstractTrees
using DataStructures

import ParserCombinator: execute, success, failure

In [2]:
@auto_hash_equals immutable EqualsOneOf{LONGEST}<:Matcher
    name::Symbol
    strings::Trie
    function EqualsOneOf(strings) 
        new(:EqualsOneOf, Trie(strings))
    end
end

EqualsOneOf(strings) = EqualsOneOf{true}(strings)

always_print(::EqualsOneOf) = true

always_print (generic function with 1 method)

In [3]:
function find_matched_positions(kk::Config, strings::Trie, ii::Int)
    Task() do 
        node = strings
        while !done(kk.source, ii)
            char, ii = next(kk.source, ii)
            if haskey(node.children, char)
                node = node.children[char]
                if node.is_key
                    produce(ii-1) #String actually ended the index before
                end
            else
                break
            end
        end
    end
end


order_match_positions(mm::EqualsOneOf{true}, iis)::Vector{Int} = reverse(collect(iis))
order_match_positions(mm::EqualsOneOf{false}, iis)::Vector{Int} = collect(iis) # I wish this was lazy

order_match_positions (generic function with 2 methods)

In [4]:
@auto_hash_equals immutable EqualsOneOfState{I,S}<:State
    start_pos::Int #Start of the matchs
    matches_end_iis::I #Iterator of matched strings 
    matches_state::S  # State for that iterator
end

In [5]:
function execute(kk::Config, mm::EqualsOneOf, ss::Clean, ii::Int)
    start_pos=ii
    found_end_pos = find_matched_positions(kk, mm.strings, ii)
    match_positions = order_match_positions(mm, found_end_pos)
    child_state = EqualsOneOfState(start_pos, match_positions, start(match_positions))
    execute(kk, mm, child_state, start_pos)
end

function execute(kk::Config, mm::EqualsOneOf, ss::EqualsOneOfState, ii)
    if done(ss.matches_end_iis, ss.matches_state)
        FAILURE
    else
        end_pos, next_matches_state = next(ss.matches_end_iis, ss.matches_state)
        next_ss = EqualsOneOfState(ss.start_pos, ss.matches_end_iis, next_matches_state)
        item = SubString(kk.source, ss.start_pos, end_pos)
        results = Any[item]
        source_state = end_pos+1
        Success(next_ss, source_state, results)
    end
end

execute (generic function with 42 methods)

In [6]:
@testset "EqualsOneOf" begin
    @test parse_one("a", EqualsOneOf(["a","b","c"])) == ["a"]
    @test parse_one("abc", EqualsOneOf(["a","b","c"])) == ["a"]
    @test parse_one("x a", E"x " + EqualsOneOf(["a","b","c"])) == ["a"]
    @test_throws ParserException parse_one("z", EqualsOneOf(["a","b","c"]))
    
    
    @test parse_all("abc", EqualsOneOf(["abc","ab","a"])) |> collect == [["abc"], ["ab"], ["a"]]
    @test parse_all("abc", EqualsOneOf{false}(["abc","ab","a"])) |> collect == [["a"], ["ab"], ["abc"]]
    
    @test parse_one("abcd", EqualsOneOf(["abc","ab","a"]) + e"cd") == ["ab", "cd"] #Requires backtracking
    
    
    # Detailed check of endpoint finding behavour
    strings = ["abc", "aabbc", "aabbcc", "aaabc", "aax"];

    @test_throws ParserException parse_one("ab", EqualsOneOf{false}(strings))
    @test parse_one("abc", EqualsOneOf{false}(strings))  == ["abc"]
    @test parse_one("abcx", EqualsOneOf{false}(strings))  == ["abc"]
    @test parse_one("aabbcc", EqualsOneOf{false}(strings))  == ["aabbc"]
    @test parse_one("aabbcx", EqualsOneOf{false}(strings))  == ["aabbc"]


    @test_throws ParserException parse_one("ab", EqualsOneOf(strings))
    @test parse_one("abc", EqualsOneOf(strings))  == ["abc"]
    @test parse_one("abcx", EqualsOneOf(strings))  == ["abc"]
    @test parse_one("aabbcc", EqualsOneOf(strings))  == ["aabbcc"]
    @test parse_one("aabbcx", EqualsOneOf(strings))  == ["aabbc"]
end;

Test Summary: | Pass  Total
  EqualsOneOf |   17     17


In [7]:
#TODO: Work out a way to rewrite the find_matched_positions to be an iterator, instread of a task
# Coss Ref: http://stackoverflow.com/questions/41072425/better-way-than-using-task-produce-consume-for-lazy-collections-express-as-cor
immutable FindMatchedPositionsIter{}
    source::I
    strings::Trie
    source_start_state::Int
end

immutable FindMatchedPositionsIterState
    source_state::Int
    node::Trie
end

Base.iteratoreltype(::Type{FindMatchedPositionsIter}) = Base.HasEltype()
Base.eltype(::Type{FindMatchedPositionsIter}) = Int
Base.iteratorsize(::Type{FindMatchedPositionsIter}) = Base.SizeUnknown()


LoadError: LoadError: TypeError: FindMatchedPositionsIter: in type definition, expected Type{T}, got UniformScaling{Int64}
while loading In[7], in expression starting on line 3